# Imersão Dev Agentes de IA Google

## Aula 1

Instalando as dependências necessárias.

In [ ]:
# -q pra esconder os textos
%pip install -q --upgrade langchain langchain-google-genai google-generativeai
%pip install -q --upgrade dotenv

Importando modelo e a chave da API.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

try:
  from google.colab import userdata # type: ignore
  GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

  print("Rodando no Google Colab. Chave da API carregado de userdata.")
except ImportError:

  load_dotenv('.env')
  GOOGLE_API_KEY = os.getenv('GEMINI_API_KEY')

  if GOOGLE_API_KEY is not None:
    print("Rodando localmente. Chave da API carregada de arquivo .env.")
  else:
    raise ValueError("GEMINI_API_KEY não encontrada.")

Conexão com Gemini.

In [ ]:
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0.0,
    google_api_key=GOOGLE_API_KEY
)

Aqui podemos fazer um teste de temperatura do modelo para a seguinte pergunta. Pode ser testado, por exemplo, o modelo com as temperaturas 0.0 e 1.0.

In [ ]:
resposta_teste = llm.invoke("Quem é você? Seja criativo!")
print(resposta_teste.content)

Um prompt para o nosso modelo de triagem. Seria uma mensagem que o próprio sistema passa para o modelo.

In [ ]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."

    """
    decisao: AUTO_RESOLVER,
    urgencia: BAIXA,
    campos_faltantes: []
    """
)


Criação de uma estrutura de dados/formatação para o modelo de triagem utilizar.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER","PEDIR_INFO","ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

Criação do modelo de triagem.

In [ ]:
llm_triagem = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0.0,
    google_api_key=GOOGLE_API_KEY
)

Criação do método de triagem, unindo mensagem de usuário e mensagem do sistema.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem:str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke(
    [
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
    ],
  )

  return saida.model_dump()

E aqui nós criamos algumas mensagens que o usuário pode mandar para o modelo.

In [ ]:
testes = [
    "Posso reembolsar a internet?",
    "Quero mais 5 dias de trabalho remoto. Como faço?",
    "Será que eu posso reembolsar recursos ou treinamentos da Alura?",
    "Quantas capivaras tem no Rio Pinheiros?"
]

Executar o teste:

In [ ]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n-> Resposta: {triagem(msg_teste)}\n")


## Aula 2

*   Pymupdf: Ler pdfs
*   Langchain_text_splitters: Separar o texto em pedaços menores
*   Faiss CPU:
*   Langchain_community: Separar o texto em pedaços menores

In [ ]:
%pip install -q --upgrade langchain_community faiss_cpu langchain_text_splitters pymupdf

Para essa aula, serão necessários 3 pdfs. Eles devem ser importados para esse ambiente (para que o modelo venha acessá-los).

Leitura dos pdfs:

*   Política de Reembolsos (Viagens e Despesas).pdf
*   Política de Uso de E-mail e Segurança da Informação.pdf
*   Políticas de Home Office.pdf

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

try:
    import google.colab # type: ignore
    pdf_path = Path('/content/')

    print("Rodando no Google Colab. Usano pasta '/content/' para acessar pdfs.")
except ImportError:
    # Assuming the PDFs are in a 'data' folder in the local environment
    pdf_path = Path('./arquivos/')
    print("Rodando localmente. Usando pasta './arquivos' para acessar pdfs.")

for doc in pdf_path.glob('*.pdf'):
  try:
    loader = PyMuPDFLoader(str(doc))
    docs.extend(loader.load())
    print(f"Carregado arquivo '{doc.name}' com sucesso!")
  except Exception as e:
    print(f"Erro ao carregar arquivo '{doc.name}': Erro {e}")

print(f'Total documentos carregados: {len(docs)}')

Dividindo o documento em 'chunks'. A separação não é bruta, pois há um "overlap" para que os chunks não fiquem "quebrados no meio", perdendo o sentido/contexto.

Também é possível fazer essa separação manualmente (Ex.: usando pandas para separar por parágrafos cada chunk).

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Tamanho "padrão" do chunk_size e chunk_overlap. 10% para o overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

Conteúdo do chunk:

In [ ]:
print(f'Total de chunks: {len(chunks)}\n')

print('------------------------------------------------------------------------------------------')
for chunk in chunks:
  print(f"{chunk.page_content}\n")
  print('-------------------------------------------------------------------------------------')

Embeddings

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [ ]:
from re import search
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

# 0.3 é um padrão
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.3, "k": 4}
)

Sugestão de atividade dos instrutores: mudar o prompt to sistema (Ex.: mudar nome da empresa, simular respostas).

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages(
    [
        ("system",
        "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
        "Responda SOMENTE com base no contexto fornecido. "
        "Se não houver base suficiente, responda apenas 'Não sei'."),

        ("human", "Pergunta: {input}\n\nContexto:\n{context}")
    ]
)

document_chain = create_stuff_documents_chain(
    llm_triagem,
    prompt_rag
)

Formatador no terminal (para ficar mais lindo e apresentável.

In [ ]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [ ]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
  docs_relacionados = retriever.invoke(pergunta)

  # Caso não houver nenhum documento, a gente nem para o modelo mandaremos
  if not docs_relacionados:
    return {
        "answer": "Não sei.",
        "citacoes": [],
        "contexto_encontrado": False
    }

  answer = document_chain.invoke({
      "input": pergunta,
      "context": docs_relacionados
    })

  txt = (answer or "").strip()

  # Caso ele já responder não sei, já retornamos uma resposta de "Não sei"
  if txt.rsplit(".!?") == "Não sei":
    return {
        "answer": "Não sei.",
        "citacoes": [],
        "contexto_encontrado": False
    }

  return {
    "answer": txt,
    "citacoes": formatar_citacoes(docs_relacionados, pergunta),
    "contexto_encontrado": True
  }

In [ ]:
testes = [
    "Posso reembolsar a internet?",
    "Quero mais 5 dias de trabalho remoto. Como faço?",
    "Será que eu posso reembolsar recursos ou treinamentos da Alura?",
    "Quantas capivaras tem no Rio Pinheiros?"
]

In [ ]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

## Aula 3

Instalando LangGraph.

In [ ]:
%pip install -q --upgrade langgraph

In [ ]:
from typing import TypedDict, Optional

class AgentState(TypedDict, total=False):
  pergunta: str
  triagem: Dict
  resposta: Optional[str]
  citacoes: List[dict]
  rag_sucesso: bool
  acao_final: str

Criação das funções que representam os nós do agente.

In [ ]:
def node_triagem(state: AgentState) -> AgentState:
  print("Executando nó de triagem...")
  resultado_triagem = triagem(state["pergunta"])
  return {
    "triagem": resultado_triagem
  }

In [ ]:
def node_auto_resolver(state: AgentState) -> AgentState:
  print("Executando nó de auto resolver...")

  resposta_rag = perguntar_politica_RAG(state["pergunta"])

  update: AgentState = {
      "resposta": resposta_rag.get("answer"), # Corrected key
      "citacoes": resposta_rag.get("citacoes",[]), # Corrected key and added default value
      "rag_sucesso": resposta_rag.get("contexto_encontrado"), # Corrected key
  }

  if resposta_rag.get("contexto_encontrado"): # Corrected key
    update["acao_final"] = "AUTO_RESOLVER"

  return update

In [ ]:
def node_pedir_info(state: AgentState) -> AgentState:
  print("Executando nó de pedir informações...")

  faltantes = state["triagem"].get("campos_faltantes",[])

  detalhe = ",".join(faltantes) if faltantes else "Tema e contexto específico"

  return {
      "resposta": f"Para avançar, preciso que detalhe: {detalhe}",
      "citacoes": [],
      "acao_final": "PEDIR_INFO"
  }

In [ ]:
def node_abrir_chamado(state: AgentState) -> AgentState:
  print("Executando nó de abrir chamado...")

  triagem = state["triagem"]

  return {
      "resposta": f"Abrindo chamado com urgência {triagem['urgencia']}. Descrição: {state['pergunta'][:140]}",
      "acao_final": "ABRIR_CHAMADO",
  }

In [ ]:
# Dá para colocar mais palavras ainda
# Serve como uma ajuda para a LLM
KEYWORDS_ABRIR_TICKET = ["aprovação", "exceção", "liberação", "abrir ticket", "abrir chamado", "acesso especial"]

def decidir_pos_triagem(state: AgentState) -> str:
    print("Decidindo após a triagem...")
    decisao = state["triagem"]["decisao"]

    match decisao:
      case "AUTO_RESOLVER":
        return "auto"
      case "PEDIR_INFO":
        return "info"
      case "ABRIR_CHAMADO":
        return "chamado"

In [ ]:
def decidir_pos_auto_resolver(state: AgentState) -> str:
  print("Decidindo após o auto resolver...")

  if state.get("rag_sucesso"):
    print("Rag com sucesso, finalizando o fluxo...\n")
    return "ok"

  state_da_pergunta = (state["pergunta" or ""]).lower()

  if any(k in state_da_pergunta for k in KEYWORDS_ABRIR_TICKET):
    print("Rag falhou, mas palavras-chave encontradas. Redirecionando para abertura de ticket.")
    return "abrir_chamado"

  print("Rag falhou, vou pedir mais informações...")
  return "pedir_info"

In [ ]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(AgentState)

workflow.add_node("triagem", node_triagem)
workflow.add_node("auto_resolver", node_auto_resolver)
workflow.add_node("pedir_info", node_pedir_info)
workflow.add_node("abrir_chamado", node_abrir_chamado)

workflow.add_edge(START, "triagem")
workflow.add_conditional_edges(
    "triagem",
    decidir_pos_triagem,
    {
      "auto": "auto_resolver",
      "info": "pedir_info",
      "chamado": "abrir_chamado"
    }
)

workflow.add_conditional_edges(
    "auto_resolver",
    decidir_pos_auto_resolver,
    {
      "info": "pedir_info",
      "chamado": "abrir_chamado",
      "ok": END
    }
)

workflow.add_edge("pedir_info", END)
workflow.add_edge("abrir_chamado", END)

grafo = workflow.compile()

In [ ]:
from IPython.display import display, Image

graph_bytes = grafo.get_graph().draw_mermaid_png()
display(Image(graph_bytes))

In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "É possível reembolsar certificações do Google Cloud?",
          "Posso obter o Google Gemini de graça?",
          "Qual é a palavra-chave da aula de hoje?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [ ]:
for msg_test in testes:
    resposta_final = grafo.invoke({"pergunta": msg_test})

    triag = resposta_final.get("triagem", {})
    print(f"PERGUNTA: {msg_test}")
    print(f"DECISÃO: {triag.get('decisao')} | URGÊNCIA: {triag.get('urgencia')} | AÇÃO FINAL: {resposta_final.get('acao_final')}")
    print(f"RESPOSTA: {resposta_final.get('resposta')}")
    if resposta_final.get("citacoes"):
        print("CITAÇÕES:")
        for citacao in resposta_final.get("citacoes"):
            print(f" - Documento: {citacao['documento']}, Página: {citacao['pagina']}")
            print(f"   Trecho: {citacao['trecho']}")

    print("------------------------------------")